In [5]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

%matplotlib inline

In [2]:
!ls Data/

311_Citizen_Requests_for_Service.csv CIN_comments_cut.csv
BR_categories_grouped.csv            Complaint_types - Master_sheet.tsv
BR_comments_cut.csv                  chattanooga_311_Service_Requests.csv
CHI_categories_grouped.csv           chicago_comments_cut.csv
CIN_categories_grouped.csv


# Create dictionary for reclassifying categories

In [12]:
df_reclass=pd.read_csv('Data/Complaint_types - Master_sheet.tsv',sep='\t')
df_reclass['CATEGORY_MAIN']=df_reclass['CATEGORY_MAIN'].map(lambda x:x.upper())
df_reclass['CATEGORY_SUB']=df_reclass['CATEGORY_SUB'].map(lambda x:x.upper())

In [13]:
df_reclass.head(2)

,CITY,DEPT_311,CODE_311,COUNT,CATEGORY_MAIN,CATEGORY_SUB
0,US_CHICAGO,health_environment,abandoned site,87,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE
1,US_CHICAGO,health_environment,air pollution work order,17839,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION


# Support functions

In [186]:
def lookup_311_code(dic,city,dept_311,code_311):
    
    vals=dic.get((city,dept_311,code_311))
    
    if vals is not None:
        return vals
    else:
        return ('unknown','unknown')


## Create the dictionary

In [189]:
reclass_dict=defaultdict(dict)


for index,row in df_reclass.iterrows():

    reclass_dict[(row['CITY'],row['DEPT_311'],row['CODE_311'])]=(row['CATEGORY_MAIN'],row['CATEGORY_SUB'])

In [190]:
reclass_dict

defaultdict(dict,
            {('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'DIRTY OR UNFENCED SWIMMING POOL'): ('HOUSING',
              'HOUSING_SAFETY'),
             ('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY'): ('GOVERNANCE',
              'GOVERNANCE_SIGNAGE'),
             ('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'JUNK, INOPERABLE, OR ABANDONED VEHICLE'): ('ENVIRONMENT',
              'ENVIRONMENT_ABANDONED_VEHICLE'),
             ('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY'): ('HOUSING',
              'HOUSING_HEALTH_CODE'),
             ('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'PROPERTY MISSING WINDOWS OR DOORS'): ('HOUSING',
              'HOUSING_GENERAL'),
             ('US_BATON_ROUGE',
              'BLIGHTED PROPERTIES',
              'PROPERTY NEEDS TO 

# DATA CLEAN AND JOIN

In [191]:
!ls Data

311_Citizen_Requests_for_Service.csv CIN_comments_cut.csv
BR_categories_grouped.csv            Complaint_types - Master_sheet.tsv
BR_comments_cut.csv                  chattanooga_311_Service_Requests.csv
CHI_categories_grouped.csv           chicago_comments_cut.csv
CIN_categories_grouped.csv


## Chicago

In [203]:
df_chicago=pd.read_csv('Data/chicago_comments_cut.csv', sep='\t')
df_chicago.drop(labels=['Unnamed: 0','INSPECTOR','locality'],axis=1,inplace=True)
df_chicago.head(2)

,COMPLAINT ID,DATA SOURCE,COMPLAINT TYPE,COMPLAINT DATE,COMPLAINT DETAIL,INSPECTION LOG
0,1725,HISTORIC DEPT. OF ENVIRONMENT,permits issued by doe work order,10/03/2011,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...
1,600301602,DEPT. OF PUBLIC HEALTH,illegal dumping work order,07/14/2017,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...


In [204]:
df_chicago['DATA SOURCE']='health_environment'
df_chicago['CITY']='US_CHICAGO'
df_chicago['COMPLAINT ID']= df_chicago['CITY'] + '_' + df_chicago['COMPLAINT ID']
df_chicago.columns=['COMPLAINT ID','DEPT_311','CODE_311','COMPLAINT DATE','COMPLAINT_1','COMPLAINT_2','CITY']
df_chicago=df_chicago[['COMPLAINT ID','CITY','DEPT_311','CODE_311','COMPLAINT_1','COMPLAINT_2','COMPLAINT DATE']]
df_chicago.head(2)

,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE
0,US_CHICAGO_1725,US_CHICAGO,health_environment,permits issued by doe work order,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,10/03/2011
1,US_CHICAGO_600301602,US_CHICAGO,health_environment,illegal dumping work order,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,07/14/2017


In [212]:
df_chicago['CATEGORY_MAIN']=df_chicago.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[0],axis=1)



In [214]:
df_chicago['CATEGORY_SUB']=df_chicago.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[1],axis=1)



In [217]:
df_chicago.to_csv('Data/US_CHICAGO_trainingdata.tsv',sep='\t')


## Baton Rouge

In [237]:
df_batonrouge=pd.read_csv('Data/BR_comments_cut.csv')
display(df_batonrouge.head(2))

df_batonrouge.drop(labels=['Unnamed: 0','DEPARTMENT','DIVISION','GEOLOCATION'],axis=1,inplace=True)
df_batonrouge['CITY']='US_BATON_ROUGE'
df_batonrouge.columns=['COMPLAINT ID','COMPLAINT DATE','DEPT_311','CODE_311','COMPLAINT_1','CITY']
df_batonrouge['COMPLAINT ID']=df_batonrouge['CITY']+ '_' + df_batonrouge['COMPLAINT ID'].map(str)
df_batonrouge['COMPLAINT_2']=None
df_batonrouge=df_batonrouge[['COMPLAINT ID','CITY','DEPT_311','CODE_311','COMPLAINT_1','COMPLAINT_2','COMPLAINT DATE']]
df_batonrouge.head(2)

,Unnamed: 0,SERVICE REQUEST ID,CREATE DATE,PARENT TYPE,TYPE,DEPARTMENT,DIVISION,COMMENTS,GEOLOCATION
0,0,220943,03/22/2016 04:40:00 PM,ROAD MAINTENANCE ISSUES,POTHOLE,MAINTENANCE,STREET MAINTENANCE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,"(30.342981, -91.114136)"
1,1,248677,07/27/2016 02:45:00 PM,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,TRANSPORTATION AND DEVELOPMENT,TRAFFIC ENGINEERING,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,"(30.439724, -91.174988)"


,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE
0,US_BATON_ROUGE_220943,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,None,03/22/2016 04:40:00 PM
1,US_BATON_ROUGE_248677,US_BATON_ROUGE,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,None,07/27/2016 02:45:00 PM


In [239]:
df_batonrouge['CATEGORY_MAIN']=df_batonrouge.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[0],axis=1)


In [240]:
df_batonrouge['CATEGORY_SUB']=df_batonrouge.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[1],axis=1)

df_batonrouge.head(2)

,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,US_BATON_ROUGE_220943,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,None,03/22/2016 04:40:00 PM,STREET,STREET_REPAIR
1,US_BATON_ROUGE_248677,US_BATON_ROUGE,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,None,07/27/2016 02:45:00 PM,STREET,STREET_REPAIR


In [244]:
df_batonrouge.to_csv('Data/US_BATON_ROUGE_trainingdata.tsv',sep='\t')

## COMBINE THEM

In [243]:
display(df_chicago.columns)==display(df_batonrouge.columns)

Index(['COMPLAINT ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

Index(['COMPLAINT ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

True

In [245]:
df_combined=pd.concat([df_chicago,df_batonrouge])

In [247]:
display(df_chicago.shape)
display(df_batonrouge.shape)
df_combined.shape

(48579, 9)

(232537, 9)

(281116, 9)

In [248]:
df_combined.to_csv('Data/US_CHI_BR_trainingdata.tsv',sep='\t')

In [249]:
display(df_combined.head(2))
display(df_combined.tail(2))

,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,US_CHICAGO_1725,US_CHICAGO,health_environment,permits issued by doe work order,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,10/03/2011,ENVIRONMENT,ENVIRONMENT_GENERAL
1,US_CHICAGO_600301602,US_CHICAGO,health_environment,illegal dumping work order,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,07/14/2017,ENVIRONMENT,ENVIRONMENT_DUMPING


,COMPLAINT ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
232535,US_BATON_ROUGE_383814,US_BATON_ROUGE,BLIGHTED PROPERTIES,PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN,VACANT HOUSE TO THE LEFT OF 5523 PLANK ROAD NE...,None,03/10/2018 10:08:00 PM,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE
232536,US_BATON_ROUGE_383818,US_BATON_ROUGE,RECYCLING,NEW RESIDENT RECYCLING CART REQUEST,I AM JUST MOVING TO THIS ADDRESS AND THERE IS ...,None,03/10/2018 10:53:00 PM,ENVIRONMENT,ENVIRONMENT_RECYCLING


# APPENDIX OF SCRATCH WORK

In [105]:

reclass_dict=defaultdict(list)


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dict[row['CITY']].append({\
        row['DEPT_311']:{row['CODE_311']:\
                         {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}})  

    
reclass_dict 

defaultdict(list,
            {'US_BATON_ROUGE': [{'BLIGHTED PROPERTIES': {'DIRTY OR UNFENCED SWIMMING POOL': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_SAFETY'}}},
              {'BLIGHTED PROPERTIES': {'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY': {'CATEGORY_MAIN': 'GOVERNANCE',
                 'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, INOPERABLE, OR ABANDONED VEHICLE': {'CATEGORY_MAIN': 'ENVIRONMENT',
                 'CATEGORY_SUB': 'ENVIRONMENT_ABANDONED_VEHICLE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_HEALTH_CODE'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY MISSING WINDOWS OR DOORS': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_GENERAL'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN': {'CATEGORY_MAIN': 'ENVIRONMENT',


In [142]:
reclass_dict=defaultdict(dict)


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    dummy_dict={'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}
    reclass_dict.update({row['CITY']:{row['DEPT_311']:{row['CODE_311']:dummy_dict}}})  

    
reclass_dict 

defaultdict(dict,
            {'US_BATON_ROUGE': {'STREET/TRAFFIC ISSUES': {'SCHOOL ZONE/SCHOOL FLASHER ISSUE': {'CATEGORY_MAIN': 'GOVERNANCE',
                'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
             'US_CHICAGO': {'health_environment': {'other': {'CATEGORY_MAIN': 'ENVIRONMENT',
                'CATEGORY_SUB': 'ENVIRONMENT_GENERAL'}}}})

In [146]:
df_reclass.groupby(['CITY','DEPT_311','CODE_311']).apply(lambda x: [dict(zip(x.CATEGORY_MAIN, x.CATEGORY_SUB))]).to_dict()

{('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'DIRTY OR UNFENCED SWIMMING POOL'): [{'HOUSING': 'HOUSING_SAFETY'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY'): [{'GOVERNANCE': 'GOVERNANCE_SIGNAGE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'JUNK, INOPERABLE, OR ABANDONED VEHICLE'): [{'ENVIRONMENT': 'ENVIRONMENT_ABANDONED_VEHICLE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY'): [{'HOUSING': 'HOUSING_HEALTH_CODE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'PROPERTY MISSING WINDOWS OR DOORS'): [{'HOUSING': 'HOUSING_GENERAL'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN'): [{'ENVIRONMENT': 'ENVIRONMENT_ABANDONED_SITE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'TALL GRASS'): [{'ENVIRONMENT': 'ENVIRONMENT_OVERGROWTH'}],
 ('US_BATON_ROUGE',
  'CODE VIOLATIONS',
  'BUILDING CODE VIOLATIONS'): [{'PLANNING': 'PLANNING_UNSAFE_ENVIRO

In [177]:
dummy=defaultdict(dict)

dummy[('US_CHICAGO','health_environment','other')]=('a','b')
dummy[('US_CHICAGO','health_environment','service stations/storage tanks work order')]=('a','c')

display(dummy)
display(dummy[('US_CHICAGO','health_environment','other')])
display(dummy.get(('US_CHICAGO','health_environment','service stations/storage tanks work order'))[1])
#display(dummy.get(('US_CHICAGO','health_environment','recycling work order'))[1])

defaultdict(dict,
            {('US_CHICAGO', 'health_environment', 'other'): ('a', 'b'),
             ('US_CHICAGO',
              'health_environment',
              'service stations/storage tanks work order'): ('a', 'c')})

('a', 'b')

'c'

In [185]:
def lookup_311_code(dic,city,dept_311,code_311):
    
    vals=dic.get((city,dept_311,code_311))
    
    if vals is not None:
        return vals
    else:
        return ('unknown','unknown')
    


lookup_311_code(dummy,'US_CHICAGO','health_environment','other')[0]

'a'

In [115]:

reclass_dictlist=list()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dictlist.append({row['CITY']:\
        {row['DEPT_311']:{row['CODE_311']:\
        {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}}})  

    
reclass_dict 

defaultdict(list,
            {'US_BATON_ROUGE': [{'BLIGHTED PROPERTIES': {'DIRTY OR UNFENCED SWIMMING POOL': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_SAFETY'}}},
              {'BLIGHTED PROPERTIES': {'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY': {'CATEGORY_MAIN': 'GOVERNANCE',
                 'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, INOPERABLE, OR ABANDONED VEHICLE': {'CATEGORY_MAIN': 'ENVIRONMENT',
                 'CATEGORY_SUB': 'ENVIRONMENT_ABANDONED_VEHICLE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_HEALTH_CODE'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY MISSING WINDOWS OR DOORS': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_GENERAL'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN': {'CATEGORY_MAIN': 'ENVIRONMENT',


In [118]:
reclass_dictlist[0]
reclass_dictlist.index(['US_CHICAGO']['health_environment']['abandoned site'])

TypeError: list indices must be integers or slices, not str

In [102]:

reclass_dict=dict()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dict=dict(reclass_dict,**{row['CITY']:{
        row['DEPT_311']:{row['CODE_311']:\
                         {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}}}  )

    
reclass_dict 

{'US_BATON_ROUGE': {'STREET/TRAFFIC ISSUES': {'SCHOOL ZONE/SCHOOL FLASHER ISSUE': {'CATEGORY_MAIN': 'GOVERNANCE',
    'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
 'US_CHICAGO': {'health_environment': {'other': {'CATEGORY_MAIN': 'ENVIRONMENT',
    'CATEGORY_SUB': 'ENVIRONMENT_GENERAL'}}}}

In [90]:
from collections import Counter
reclass_dict=dict()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    dummydict={'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}
    reclass_dict+={row['CITY']:{row['DEPT_311']:{row['CODE_311']}}}
    

    
reclass_dict 

TypeError: unsupported operand type(s) for +=: 'dict' and 'dict'